# Model Train

## Imports

In [1]:
from sklearn.model_selection import train_test_split
from src.model import DistilBertForClassification
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer
from src.data_loader import Custom_Dataset_Loader
from torch.utils.data import DataLoader
from torch.optim import AdamW
from pathlib import Path
import pandas as pd
import torch
import os

pd.set_option("display.max_colwidth", None)

## Functions

In [2]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()
    for batch, data in enumerate(dataloader):

        X = data["input_ids"].to(device)
        attention_masks = data["attention_mask"].to(device)
        y = data["label"].to(device)

        pred = model(X, attention_masks)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):

    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for data in dataloader:
            X = data["input_ids"].to(device)
            attention_masks = data["attention_mask"].to(device)
            y = data["label"].to(device)

            pred = model(X, attention_masks)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

## Initial Variables

### Model Hyperparameters and save dir


In [3]:
output_dir = Path("..", "data", "models")
batch_size = 256
epochs = 10
lr = 1e-5

### Dataset Load

In [ ]:
df = pd.read_csv(Path("..", "data", "datasets", "filtered_amazon_data.csv"))
df.head()

### Define Device

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

## Data Preparation

### Prompt Creation

In [6]:
df["feature_concat"] = (
    "Title: "
    + df["title"]
    + " [SEP] "
    + "Brand: "
    + df["brand"]
    + " [SEP] "
    + "Description: "
    + df["description"]
    + " [SEP] "
    + "Features: "
    + df["feature"]
)

### Label Encoding

In [7]:
label_encoder = LabelEncoder()
df["main_cat"] = label_encoder.fit_transform(df["main_cat"].to_list())
classes = df["main_cat"].unique()

### Data Split

In [8]:
X = df["feature_concat"]
y = df["main_cat"]
X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train, X_test, Y_train, Y_test = (
    X_train.reset_index(drop=True),
    X_test.reset_index(drop=True),
    Y_train.reset_index(drop=True),
    Y_test.reset_index(drop=True),
)

### Data Tokenization

In [9]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_dataset = Custom_Dataset_Loader(X_train, Y_train, tokenizer)
test_dataset = Custom_Dataset_Loader(X_test, Y_test, tokenizer)

In [11]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

## Model Definition

In [12]:
model = DistilBertForClassification(num_classes=len(classes))

In [13]:
optimizer = AdamW(model.parameters(), lr=lr)
loss_ce = torch.nn.CrossEntropyLoss()

In [ ]:
model.to(device)

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_ce, optimizer)
    test_loop(test_loader, model, loss_ce)
print("Training Finished")

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

torch.save(model.state_dict(), Path(output_dir, "New_Amazon_Product_Classifier.pth"))